# Fully Convolutional Neural Networks

Objectives:
- Load a CNN model pre-trained on ImageNet
- Transform the network into a Fully Convolutional Network 
- Apply the network perform weak segmentation on images

In [ ]:
%matplotlib inline
import numpy as np
from scipy.misc import imread, imresize
import matplotlib.pyplot as plt

In [ ]:
# Load a pre-trained ResNet50
# We use include_top = False for now,
# as we'll import output Dense Layer later

from keras.applications.resnet50 import ResNet50
base_model = ResNet50(include_top=False)

print(base_model.output_shape)

In [ ]:
# print(base_model.summary())

In [ ]:
res5c = base_model.layers[-2]
type(res5c)

In [ ]:
res5c.output_shape

In [ ]:
avg_pool = base_model.layers[-1]
type(avg_pool)

In [ ]:
avg_pool.pool_size, avg_pool.strides

In [ ]:
avg_pool.output_shape

### Fully convolutional ResNet

- Out of the `res5c` residual block, the resnet outputs a tensor of shape $H \times W \times 2048$. 
- For the default ImageNet input, $224 \times 224$, the output size is $7 \times 7 \times 2048$
- After this bloc, the ResNet uses an average pooling `AveragePooling2D(pool_size=(7, 7))` with `(7, 7)` strides which divides by 7 the width and height

#### Regular ResNet layers 

The regular ResNet head after the base model is as follows: 
```py
x = base_model.output
x = Flatten()(x)
x = Dense(1000)(x)
x = Softmax()(x)
```

Here is the full definition of the model: https://github.com/fchollet/keras/blob/master/keras/applications/resnet50.py

#### Our Version

- To keep spatial information as much as possible, we will remove the average pooling.
- We want to retrieve the labels information, which is stored in the Dense layer. We will load these weights afterwards
- We will change the Dense Layer to a Convolution layer to keep spatial information, to output a $H \times W \times 1000$.
- We want to apply a softmax only on the last dimension

#### A custom Softmax

We build the following Custom Layer to apply a softmax only to the last dimension of a tensor:

In [ ]:
import keras
from keras.engine import Layer
import keras.backend as K

# A custom layer in Keras must implement the four following functions:
class SoftmaxMap(Layer):
    # Init function
    def __init__(self, axis=-1,**kwargs):
        self.axis=axis
        super(SoftmaxMap, self).__init__(**kwargs)

    # There's no parameters, so we don't need this one
    def build(self,input_shape):
        pass

    # This is the layer we're interested in. 
    # Very similar to the regular softmax
    def call(self, x, mask=None):
        e = K.exp(x - K.max(x, axis=self.axis, keepdims=True))
        s = K.sum(e, axis=self.axis, keepdims=True)
        return e / s

    # The output shape is the same as the input shape
    def get_output_shape_for(self, input_shape):
        return input_shape


** Exercise **
- What is the shape of the convolution kernel we want to apply to replace the Dense ?
- Build the fully convolutional model as described above.
- You may use `base_model.layers` to find which layer to remove
- You may use the Keras `Convolution2D(output_channels, filter, filter)` layer

In [ ]:
from keras.layers import Convolution2D
from keras.models import Model

input = base_model.layers[0].input
# TODO
output = input

fully_conv_ResNet = Model(input = input, output = output)

In [ ]:
# %load solutions/fully_conv.py

### Loading Dense weights

- we provide the weights and bias of the last Dense layer of ResNet50 in file `weights_dense.h5`
- our last layer is now a convolutional layer instead of Dense

In [ ]:
import h5py
h5f = h5py.File('weights_dense.h5','r')
w = h5f['w'][:]
b = h5f['b'][:]
h5f.close()

In [ ]:
last_layer = fully_conv_ResNet.layers[-2]

print("loaded weight shape", w.shape)
print("conv last layer weights shape", last_layer.get_weights()[0].shape)

In [ ]:
# reshape the weights
w_reshaped = w.reshape((1,1,2048,1000))

# set the conv layer weights
last_layer.set_weights([w_reshaped, b])

### A forward pass

- We define the following function to test our new network. 
- It resizes the input to a given size, then uses `model.predict` to compute the output

In [ ]:
from scipy.misc import imread, imresize
from keras.applications.imagenet_utils import preprocess_input

def forward_pass_resize(img_path, img_size):
    img_raw = imread(img_path)
    print("img shape before resizing: "+ str(img_raw.shape))
    img = imresize(img_raw, size=img_size).astype("float32")
    img = preprocess_input(img[np.newaxis])
    print("img batch size shape before forward pass:", img.shape)
    z = fully_conv_ResNet.predict(img)
    return z

In [ ]:
output = forward_pass_resize("dog.jpg", (800,600))
print("output shape", output.shape)

### Finding dog-related classes
ImageNet uses an ontology of concepts, from which classes are derived. A synset corresponds to a node in the ontology.

For example species of dogs are children nodes of the synset `dog`

In [ ]:
# Helper file for importing synsets from imagenet
import imagenet_tool
synset = "n02084071" # synset corresponding to dogs
ids = imagenet_tool.synset_to_dfs_ids(synset)
print("all dog classes ids", ids)

### Unsupervised heatmap of the class "dog"

The following function builds a heatmap from a forward pass. It sums the representation for all ids corresponding to a synset

In [ ]:
def build_heatmap(z, synset):
    ids = imagenet_tool.synset_to_dfs_ids(synset)
    ids = np.array([id_ for id_ in ids if id_ is not None])
    x = z[0,:,:,ids].sum(axis=0)
    print("size of heatmap: " + str(x.shape))
    return x

In [ ]:
def display_img_and_heatmap(img_path, heatmap):
    dog = imread(img_path)
    plt.figure(figsize=(12, 8))
    plt.subplot(1, 2, 1)
    plt.imshow(dog)
    plt.axis('off')
    plt.subplot(1, 2, 2)
    plt.imshow(heatmap, interpolation='nearest', cmap="viridis")
    plt.axis('off')

**Exercise**
- What is the size of the heatmap compared to the input image?
- Build 3 dog heatmaps from `"dog.jpg"`, with the following sizes:
  - `(400, 640)`
  - `(800, 1280)`
  - `(1600, 2560)`
- What do you observe? 

You may plot a heatmap using the above function `display_img_and_heatmap`

In [ ]:
# dog synset
s = "n02084071"
# Todo

In [ ]:
# %load solutions/build_heatmaps.py

In [ ]:
display_img_and_heatmap("dog.jpg", heatmap_1)

### Combining the 3 heatmaps
By combining the heatmaps at different scales, we obtain a much better information about the location of the dog.

**Bonus**
- Combine the three heatmap by resizing them to a similar shape, and averaging them
- A geometric norm will work better than standard average!

In [ ]:
# %load solutions/geom_avg.py